In [1]:
import numpy as np
import pandas as pd
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import matplotlib.pyplot as plt
import emoji

from sys import path as pylib #im naming it as pylib so that we won't get confused between os.path and sys.path 
import os
pylib += [os.path.abspath(r'/home/Jay/Notebooks/Group9_emotion_detection/torchMoji')]

In [19]:
data = pd.read_csv('../../deepmoji/data/train.txt', sep = '\t')

In [3]:
label2emotion = {0: "others", 1: "happy", 2: "sad", 3: "angry"}
emotion2label = {"others": 0, "happy": 1, "sad": 2, "angry": 3}

emoticons_additional = {
    '(^・^)': '<happy>', ':‑c': '<sad>', '=‑d': '<happy>', ":'‑)": '<happy>', ':‑d': '<laugh>',
    ':‑(': '<sad>', ';‑)': '<happy>', ':‑)': '<happy>', ':\\/': '<sad>', 'd=<': '<annoyed>',
    ':‑/': '<annoyed>', ';‑]': '<happy>', '(^�^)': '<happy>', 'angru': 'angry', "d‑':":
        '<annoyed>', ":'‑(": '<sad>', ":‑[": '<annoyed>', '(�?�)': '<happy>', 'x‑d': '<laugh>',
}

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
               'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
              'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter",
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter",
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=True,  # spell correction for elongated words
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons, emoticons_additional]
)


def tokenize(text):
    text = " ".join(text_processor.pre_process_doc(text))
    return text

Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


In [4]:
x1=[]
x2=[]
x3=[]
y=[]
labels={}
i=0
for ind, row in data.iterrows():
    if row['label'] not in labels:
        labels[row['label']]=i
        i+=1
    y.append(labels[row['label']])
    x1.append(tokenize(row['turn1']))
    x2.append(tokenize(row['turn2']))
    x3.append(tokenize(row['turn3']))

In [5]:
test_data = pd.read_csv('../../deepmoji/data/test.txt', sep = '\t')
test_x1=[]
test_x2=[]
test_x3=[]
test_y=[]
test_labels={}
i=0
for ind, row in test_data.iterrows():
    if row['label'] not in test_labels:
        test_labels[row['label']]=i
        i+=1
    test_y.append(test_labels[row['label']])
    test_x1.append(tokenize(row['turn1']))
    test_x2.append(tokenize(row['turn2']))
    test_x3.append(tokenize(row['turn3']))

In [6]:
from __future__ import print_function, division, unicode_literals
import json

from torchmoji.sentence_tokenizer import SentenceTokenizer
from torchmoji.model_def import torchmoji_feature_encoding
from torchmoji.model_def import torchmoji_emojis
from torchmoji.global_variables import PRETRAINED_PATH, VOCAB_PATH

def encode_emoji(x):
    model = torchmoji_emojis(PRETRAINED_PATH)
    maxlen = 30
    batch_size = 32

    print('Tokenizing using dictionary from {}'.format(VOCAB_PATH))
    with open(VOCAB_PATH, 'r') as f:
        vocabulary = json.load(f)
    st = SentenceTokenizer(vocabulary, maxlen)
    tokenized, _, _ = st.tokenize_sentences(x)
    print('Loading model from {}.'.format(PRETRAINED_PATH))
    print(model)
    print('Running predictions.')
    
    encoding = np.zeros((len(x), 64))
    for i in range(0, len(x), 300):    
        encoding[i:i+300] = model(tokenized[i:i+300])
    return encoding

In [7]:
y=np.array(y)
idx_others=np.where(y==0)[0]
x1 = np.array(x1)
x2 = np.array(x2)
x3 = np.array(x3)

In [8]:
x_all = np.c_[x1,x2,x3]
x_all = np.array([' <eos> \n '.join(x) for x in x_all])

In [9]:
x_emoji = encode_emoji(x_all)

Tokenizing using dictionary from /home/Jay/Notebooks/Group9_emotion_detection/torchMoji/model/vocabulary.json
Loading model from /home/Jay/Notebooks/Group9_emotion_detection/torchMoji/model/pytorch_model.bin.
TorchMoji(
  (embed): Embedding(50000, 256)
  (embed_dropout): Dropout2d(p=0)
  (lstm_0): LSTMHardSigmoid(256, 512, batch_first=True, bidirectional=True)
  (lstm_1): LSTMHardSigmoid(1024, 512, batch_first=True, bidirectional=True)
  (attention_layer): Attention(2304, return attention=False)
  (final_dropout): Dropout(p=0)
  (output_layer): Sequential(
    (0): Linear(in_features=2304, out_features=64, bias=True)
    (1): Softmax()
  )
)
Running predictions.


/home/Jay/dev32/lib/python3.5/site-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [10]:
x1_emoji = encode_emoji(x1)
x2_emoji = encode_emoji(x2)
x3_emoji = encode_emoji(x3)

Tokenizing using dictionary from /home/Jay/Notebooks/Group9_emotion_detection/torchMoji/model/vocabulary.json
Loading model from /home/Jay/Notebooks/Group9_emotion_detection/torchMoji/model/pytorch_model.bin.
TorchMoji(
  (embed): Embedding(50000, 256)
  (embed_dropout): Dropout2d(p=0)
  (lstm_0): LSTMHardSigmoid(256, 512, batch_first=True, bidirectional=True)
  (lstm_1): LSTMHardSigmoid(1024, 512, batch_first=True, bidirectional=True)
  (attention_layer): Attention(2304, return attention=False)
  (final_dropout): Dropout(p=0)
  (output_layer): Sequential(
    (0): Linear(in_features=2304, out_features=64, bias=True)
    (1): Softmax()
  )
)
Running predictions.
Tokenizing using dictionary from /home/Jay/Notebooks/Group9_emotion_detection/torchMoji/model/vocabulary.json
Loading model from /home/Jay/Notebooks/Group9_emotion_detection/torchMoji/model/pytorch_model.bin.
TorchMoji(
  (embed): Embedding(50000, 256)
  (embed_dropout): Dropout2d(p=0)
  (lstm_0): LSTMHardSigmoid(256, 512, batc

In [11]:
def top_elements(array, k):
    ind = np.argpartition(array, -k)[-k:]
    return ind[np.argsort(array[ind])][::-1]

EMOJIS = ":joy: :unamused: :weary: :sob: :heart_eyes: \
:pensive: :ok_hand: :blush: :heart: :smirk: \
:grin: :notes: :flushed: :100: :sleeping: \
:relieved: :relaxed: :raised_hands: :two_hearts: :expressionless: \
:sweat_smile: :pray: :confused: :kissing_heart: :heartbeat: \
:neutral_face: :information_desk_person: :disappointed: :see_no_evil: :tired_face: \
:v: :sunglasses: :rage: :thumbsup: :cry: \
:sleepy: :yum: :triumph: :hand: :mask: \
:clap: :eyes: :gun: :persevere: :smiling_imp: \
:sweat: :broken_heart: :yellow_heart: :musical_note: :speak_no_evil: \
:wink: :skull: :confounded: :smile: :stuck_out_tongue_winking_eye: \
:angry: :no_good: :muscle: :facepunch: :purple_heart: \
:sparkling_heart: :blue_heart: :grimacing: :sparkles:".split(' ')

In [20]:
turn1=[]
turn2=[]
turn3=[]

for i, text in enumerate(data['turn1'].values):
    emoji_ids = top_elements(x1_emoji[i], 5)
    emojis = map(lambda x: EMOJIS[x], emoji_ids)
    turn1.append(emoji.emojize("{} {}".format(text,' '.join(emojis)), use_aliases=True))

for i, text in enumerate(data['turn2'].values):
    emoji_ids = top_elements(x2_emoji[i], 5)
    emojis = map(lambda x: EMOJIS[x], emoji_ids)
    turn2.append(emoji.emojize("{} {}".format(text,' '.join(emojis)), use_aliases=True))

for i, text in enumerate(data['turn3'].values):
    emoji_ids = top_elements(x3_emoji[i], 5)
    emojis = map(lambda x: EMOJIS[x], emoji_ids)
    turn3.append(emoji.emojize("{} {}".format(text,' '.join(emojis)), use_aliases=True))
    #print(emoji.emojize("{} {}".format(x1[i],' '.join(emojis)), use_aliases=True))

data['turn1'] = turn1
data['turn2'] = turn2
data['turn3'] = turn3

In [21]:
labels = []
for i, prob in enumerate(x_emoji):
    emoji_ids = top_elements(prob, 5)
    emojis = map(lambda x: EMOJIS[x], emoji_ids)
    labels.append(emoji.emojize(' '.join(emojis), use_aliases=True))
    
data['label_emo'] = labels

In [22]:
data

id                                              turn1  \
0          0                    Don't worry  I'm girl 💁 ✋ 🙅 😉 👊   
1          1                              When did I? 😳 👀 😕 😐 😔   
2          2                                       By ✌ 🎵 💔 💜 💓   
3          3                           U r ridiculous 😡 😠 😤 ✋ 😑   
4          4                       Just for time pass 🙏 😌 😏 😈 😅   
5          5                         I'm a dog person 😈 💁 😊 😎 😌   
6          6                               So whatsup 😎 😏 ✌ 😈 😜   
7          7                                       Ok 😐 👌 👍 ✌ 😕   
8          8                                  Really? 😑 😒 😐 😠 😡   
9          9                                      Bay ✌ ✋ 😘 💓 💛   
10        10                      I hate my boyfriend 😡 😠 😤 👊 😒   
11        11                         I will do night. 😴 ✌ 😌 👌 ✨   
12        12                            Sure go ahead 👍 👌 ✌ 😉 😌   
13        13                                      Bad 🎵 💔 😈 🎶 😞   
14        14                          Ok get it...... ☺ 👍 💓 🎵 💛   
15        15          Money money and lots of money😍😍 🎵 🎶 💯 😎 😈   
16        16                            My gf left ne 😢 💔 😔 😪 😞   
17        17                                 get lost 🎵 👊 ✌ ✋ 🎶   
18        18            You are lying and i know that ✋ 😴 😒 👏 ✌   
19        19                   Ur creator is very bad 😷 😈 🙅 ✋ 💀   
20        20                             ehat is hehe 🙊 😋 ☺ 🙈 😈   
21        21                            Do you dance? 🎶 🎵 😏 😉 👀   
22        22                            I hate it too 😣 😞 😔 😠 😤   
23        23                               Not always 🙅 💁 😌 😪 😒   
24        24  Bcoz u dont know wat is to miss someone 😢 💔 😞 😪 😣   
25        25                                     Yeah 👊 😏 ✌ 😎 😒   
26        26             I want to tell you something 🙊 😔 😞 😏 🙈   
27        27  Yes I will send the email and you write all yo...   
28        28                       You are very funny 😈 👏 😄 😂 😊   
29        29                                      wth 😳 😑 😐 😠 😒   
...      ...                                                ...   
30130  30130                         My spoken power  💪 🙌 👊 🙏 👏   
30131  30131                                Only 11 k ☺ 💓 ✌ 🎵 💯   
30132  30132                      So what should I do 😕 😐 😓 😔 😪   
30133  30133                              And shut up 😒 👊 😑 ✋ 😠   
30134  30134                           I am also good 😌 😎 👍 👌 😏   
30135  30135                  I am not in a good mood 😠 😡 😤 😒 🙅   
30136  30136                                      Yes 🙌 👏 😍 😌 💪   
30137  30137      And after u be mother for my child  💯 💪 😘 💛 🙌   
30138  30138                               A.R Rehman 💓 🎵 💛 🎶 💙   
30139  30139                                    Like? 👀 😳 😕 😐 😒   
30140  30140              Nope I hate to be a doctor. 🙅 😷 😖 😠 🔫   
30141  30141                                      Yep 👍 ✌ 🙌 👌 😌   
30142  30142          Can you give me those 360 pages ☺ 💓 🎵 💛 🎶   
30143  30143  Anyway, I'm sorry, it's 1AM and I have school ...   
30144  30144                 What's your day going on 👀 😌 👏 😞 😕   
30145  30145                         you can sleep 😂😂 😴 ✌ 🎵 🎶 😉   
30146  30146                                  Really😻 😑 😒 😡 😤 😠   
30147  30147                                  Nothing 🎵 🙅 😐 😑 😞   
30148  30148                       I m feeling so hot 😓 😷 😖 😣 😫   
30149  30149                              Will u come 🎵 🎶 👀 😢 😞   
30150  30150                              Shut up now ✋ 👊 😒 😠 😡   
30151  30151              I have ready for your.   Bg 💓 💛 😈 😉 ☺   
30152  30152                                    Waste 😴 😪 😒 😑 😤   
30153  30153                 How to remember anything 🎵 💔 🎶 😪 😓   
30154  30154                                   Dilute 💓 💛 💙 💪 😍   
30155  30155                             I don't work 🙅 ✋ 😑 😤 😒   
30156  30156                  I'm not getting you 😭😭😭 🙅 😡 ✋ 😠 😤   
30157  30157                                     Haha 😄 😅

In [23]:
data_others = data[data['label']=='others']

In [24]:
data_others

id                                              turn1  \
0          0                    Don't worry  I'm girl 💁 ✋ 🙅 😉 👊   
2          2                                       By ✌ 🎵 💔 💜 💓   
4          4                       Just for time pass 🙏 😌 😏 😈 😅   
5          5                         I'm a dog person 😈 💁 😊 😎 😌   
6          6                               So whatsup 😎 😏 ✌ 😈 😜   
7          7                                       Ok 😐 👌 👍 ✌ 😕   
8          8                                  Really? 😑 😒 😐 😠 😡   
9          9                                      Bay ✌ ✋ 😘 💓 💛   
11        11                         I will do night. 😴 ✌ 😌 👌 ✨   
12        12                            Sure go ahead 👍 👌 ✌ 😉 😌   
14        14                          Ok get it...... ☺ 👍 💓 🎵 💛   
20        20                             ehat is hehe 🙊 😋 ☺ 🙈 😈   
23        23                               Not always 🙅 💁 😌 😪 😒   
25        25                                     Yeah 👊 😏 ✌ 😎 😒   
27        27  Yes I will send the email and you write all yo...   
29        29                                      wth 😳 😑 😐 😠 😒   
31        31                                ok i will 😌 👌 😊 👍 😏   
35        35                                   Abt me 💁 🎵 😌 👀 😎   
36        36                                    crazy 😳 😜 😴 🎵 😅   
37        37                             Goa in India 😍 👍 🙌 😎 ✌   
38        38                            That was mean 😔 😕 😞 😢 😪   
40        40                               I don't no 🙅 ✋ 😠 😡 😑   
41        41                   Send me your nekde pic 😈 👀 😏 😉 😜   
45        45                                  Nothing 🎵 🙅 😐 😑 😞   
46        46                                 Not sure 😕 😞 😐 😬 😅   
47        47            I like your positive approach 👍 😌 👌 😊 ☺   
48        48                              You tell me 👀 😏 🎵 🙊 😳   
49        49                                     Hmmm ☺ 💓 💛 🎵 🎶   
51        51                                   About? 👀 🙊 😳 😏 😴   
53        53                              So teach me 🎵 🎶 😏 🙏 😌   
...      ...                                                ...   
30113  30113  I like your page and I also follow your page 😉...   
30114  30114                            I saved the 🌍 🙌 🙏 😁 👏 😎   
30115  30115                                     Whom 👀 💔 😳 💓 😐   
30116  30116                         How about losing 😅 😡 🔫 😤 💁   
30117  30117                 So you intentimes hehehe 🙊 😈 😏 😉 🙈   
30120  30120                                Ya It was 😏 😋 ✋ 💁 😜   
30121  30121                                   Simply ✌ 🎵 🎶 ✨ 😌   
30123  30123                             i need a job 😩 😫 😓 😤 😐   
30125  30125                             I am waiting 👀 😴 😠 😐 😪   
30127  30127                       It's a grasshopper 😌 🙏 🙌 😷 👌   
30128  30128                               I miss you 💔 😔 😞 😢 😪   
30129  30129                            I don't know  😐 😞 😕 😳 😔   
30130  30130                         My spoken power  💪 🙌 👊 🙏 👏   
30131  30131                                Only 11 k ☺ 💓 ✌ 🎵 💯   
30134  30134                           I am also good 😌 😎 👍 👌 😏   
30138  30138                               A.R Rehman 💓 🎵 💛 🎶 💙   
30139  30139                                    Like? 👀 😳 😕 😐 😒   
30141  30141                                      Yep 👍 ✌ 🙌 👌 😌   
30142  30142          Can you give me those 360 pages ☺ 💓 🎵 💛 🎶   
30143  30143  Anyway, I'm sorry, it's 1AM and I have school ...   
30144  30144                 What's your day going on 👀 😌 👏 😞 😕   
30147  30147                                  Nothing 🎵 🙅 😐 😑 😞   
30149  30149                              Will u come 🎵 🎶 👀 😢 😞   
30151  30151              I have ready for your.   Bg 💓 💛 😈 😉 ☺   
30153  30153                 How to remember anything 🎵 💔 🎶 😪 😓   
30154  30154                                   Dilute 💓 💛 💙 💪 😍   
30155  30155                             I don't work 🙅 ✋ 😑 😤 😒   
30157  30157                                     Haha 😄 😅

In [31]:
data[data['label']=='others']['label_emo']

0        ☺ 💓 🎶 💛 🎵
2        💓 🎶 🎵 💛 ✨
4        ☺ 💓 💛 ✨ 🎵
5        ☺ 💓 ✨ ✌ 💛
6        ☺ 💓 💛 😄 🎵
7        💓 💛 ✨ ☺ ❤
8        ☺ 💓 💛 👍 💔
9        💓 💛 ☺ ❤ ✨
11       ☺ 💓 💛 ✨ 🎵
12       💓 💛 ☺ ❤ 😘
14       💓 ☺ 💛 👍 ✨
20       ☺ 💓 😄 😉 😜
23       ☺ 💓 💛 😉 👍
25       💓 ☺ 💛 ✨ 👍
27       🙏 👍 😊 ✨ ☺
29       ☺ 💪 💓 😉 👊
31       ✨ 💓 💛 ☺ ✌
35       💓 💛 ☺ ✨ ❤
36       💓 ☺ 💛 🎶 🎵
37       ☺ 💓 💛 ✨ ✌
38       💓 ☺ ✨ 💛 😉
40       ☺ 💓 💛 🎶 🎵
41       ☺ 💓 💛 😄 ✌
45       ☺ 💓 💛 ✨ 😄
46       ☺ 💓 ✨ 🙏 👀
47       💓 ☺ 💛 👍 ✨
48       ☺ 💓 💛 🎶 🎵
49       💓 ☺ 💛 ✨ 🎶
51       ☺ 💓 💛 ✋ 🎶
53       💓 ☺ 🎵 💛 🎶
           ...    
30113    💓 💛 ☺ ✨ ✌
30114    💓 ✨ ☺ 💛 🎶
30115    ☺ 💓 💛 ✨ 😄
30116    👍 ☺ 💓 💛 😎
30117    💓 ☺ 💛 ✌ 😄
30120    ☺ 💓 💛 ✌ 😄
30121    💓 ☺ 💛 ✨ 🎶
30123    ☺ 💓 ✨ 💛 ✌
30125    💓 ☺ 💛 ✨ 🎶
30127    😖 😷 💓 ✨ ☺
30128    💓 💛 ☺ ❤ 😢
30129    💓 ☺ 💛 ✨ 👍
30130    💪 💓 ☺ ✨ 💛
30131    💓 ☺ 💛 ✨ 🎵
30134    ☺ 💓 💛 ✨ ✌
30138    💓 🎶 💛 ☺ ✨
30139    💓 ☺ 💛 ✨ ❤
30141    💓 ☺ 💛 ✨ 🎶
30142    ☺ 💓 💛 ✨ 🎵
30143    💓 ☺ 💛 👍 ✌
30144    ☺ 💓 ✨ 👍 💪
30147    💓 🎵